# 逐字稿自動分段工具 (2,000–3,000 字)

這份 Notebook 把貼成一整段的逐字稿，依字數切成段落。流程：

1. 設定要處理的檔案路徑，以及每段字數範圍（預設 2,000–3,000 字）。
2. 執行分段函式，先預覽每段的長度與內容。
3. 確認沒問題後寫回原檔（或另存）。

> ⚠️ 建議先備份原檔，或在 `WRITE_BACK` 參數確認無誤後再改為 `True`。


In [2]:
from pathlib import Path
Path.cwd()


In [3]:
from pathlib import Path

# === 基本參數 ===
SOURCE_PATH = Path("work") / "mkdocs/My_Notes/課_四34_所得税法四/逐字稿/W04_0925.md"

ENCODING = "utf-8"                                 # 文字編碼
MIN_CHARS = 2000                                    # 單段最少字數
MAX_CHARS = 3000                                    # 單段最多字數

# 如果想另存成新檔，將下列設成 Path 物件；若為 None 則改寫原檔。
OUTPUT_PATH = None



In [4]:
import re
from typing import List

_split_pattern = re.compile(r".*?(?:[。！？!?；;：:]\s*|\n+|$)", re.S)

def _split_sentences(text: str) -> List[str]:
    """把文字依句號、感嘆號、問號、頓號、換行等標記拆成片段。"""
    sentences = [frag for frag in _split_pattern.findall(text) if frag]
    return sentences

def split_into_chunks(text: str, min_chars: int = 2000, max_chars: int = 3000) -> List[str]:
    """依指定字數範圍，把文字拆成段落。"""
    if min_chars <= 0 or max_chars <= 0:
        raise ValueError("min_chars 和 max_chars 必須為正整數")
    if min_chars > max_chars:
        raise ValueError("min_chars 不能大於 max_chars")

    sentences = _split_sentences(text)
    chunks: List[str] = []
    current = ""

    for sentence in sentences:
        if not sentence:
            continue
        # 若單一句過長，直接硬切
        while len(sentence) > max_chars:
            need = max_chars - len(current)
            if need <= 0:
                if current:
                    chunks.append(current.rstrip())
                    current = ""
                need = max_chars
            current += sentence[:need]
            sentence = sentence[need:]
            if len(current) >= min_chars:
                chunks.append(current.rstrip())
                current = ""
        # 一般情形
        if len(current) + len(sentence) <= max_chars:
            current += sentence
        else:
            if current:
                if len(current) < min_chars:
                    needed = min(min_chars - len(current), len(sentence))
                    current += sentence[:needed]
                    sentence = sentence[needed:]
                chunks.append(current.rstrip())
            current = sentence

    if current:
        chunks.append(current.rstrip())

    return chunks



In [5]:
# === 讀檔並預覽分段結果 ===
raw_text = SOURCE_PATH.read_text(encoding=ENCODING)
chunks = split_into_chunks(raw_text, MIN_CHARS, MAX_CHARS)

print(f"原文字數：{len(raw_text)}")
print(f"段落數量：{len(chunks)}")
print(f"每段字數：{[len(chunk) for chunk in chunks[:5]]} ...\n")

# 預覽前兩段內容
for idx, chunk in enumerate(chunks[:2], start=1):
    print(f"---- 第 {idx} 段（{len(chunk)} 字）----")
    preview = chunk.replace('\r\n', '\n').replace('\r', '\n')
    snippet = preview[:200] + ('...' if len(preview) > 200 else '')
    print(snippet)
    print()



In [6]:
# === 寫回檔案 ===
WRITE_BACK = True  # 確認無誤後改成 True
TARGET_PATH = OUTPUT_PATH or SOURCE_PATH

if WRITE_BACK:
    normalized = [chunk.strip() for chunk in chunks]
    TARGET_PATH.write_text('\n\n\n\n\n\n\n\n'.join(normalized), encoding=ENCODING)
    print(f'已寫入：{TARGET_PATH}')
else:
    print('WRITE_BACK 為 False，未寫入檔案。')